# Predict Price of Airbnb Listings

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [8]:
df=pd.read_csv("train.csv",low_memory=False)

# EDA

In [15]:
# df.info()

In [21]:
df.columns[0:15]

Index(['id', 'name', 'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'host_id', 'host_name', 'host_since'],
      dtype='object')

## Numeric features

In [24]:
num_cols=df.columns[[51,52,53,54,55,56,57,63,64,46,47,48,42,44,36,37,38,39,22]]

In [28]:
# impute mean for missing values to build a base line model
dict={}
for i in num_cols:
    dict[i]=df[i].mean()

In [29]:
X=df[num_cols].fillna(value=dict)
Y=df["price"]

In [30]:
# investigatig feature importance using Lasso Regression
from sklearn import linear_model
clf = linear_model.Lasso(alpha=3)
clf.fit(X,Y)

Lasso(alpha=3)

In [31]:
clf.coef_

array([ 4.94208837e-01, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  1.73471922e+01, -4.62796330e+00,  3.28383919e-01,
       -2.38426780e+00,  9.19788516e-03,  2.79956326e-08, -2.09049989e-01,
        2.12232835e-02,  5.72266739e-01,  3.34519902e+01,  2.38020988e+01,
        5.26271563e+00,  0.00000000e+00,  6.77806341e-02])

In [32]:
yes=abs(clf.coef_)!=0
no=abs(clf.coef_)==0


 - 'review_scores_rating', -ok
 - 'reviews_per_month', -ok
 - 'number_of_reviews',-ok
 - 'square_feet', -ok
 - 'accommodates', -ok
 - 'bathrooms', -ok
 - 'bedrooms', -ok
 - 'beds',-ok
 - review_scores_cleanliness, -ok
 - guests_included -ok


In [34]:
# using ghroupby to investigate in individual features
df.groupby("review_scores_location")["price"].count()

review_scores_location
2.0        34
4.0        12
5.0        14
6.0       177
7.0       217
8.0      1674
9.0      7780
10.0    15922
Name: price, dtype: int64

In [35]:
df.groupby(pd.cut(df["review_scores_rating"], bins=10))["price"].mean()

review_scores_rating
(19.92, 28.0]    133.125000
(28.0, 36.0]     109.000000
(36.0, 44.0]     120.500000
(44.0, 52.0]     127.966667
(52.0, 60.0]     119.545817
(60.0, 68.0]     161.333333
(68.0, 76.0]     130.012461
(76.0, 84.0]     122.938192
(84.0, 92.0]     132.338369
(92.0, 100.0]    142.479895
Name: price, dtype: float64

## Text features

In [36]:
#df.columns

In [ ]:
# creating new columnes using text data 
v = CountVectorizer(max_features=55)
v.fit_transform(df["description"])
v.get_feature_names_out()

#array(['access', 'all', 'and', 'apartment', 'are', 'as', 'away',
       #'bathroom', 'be', 'bed', 'bedroom', 'brooklyn', 'building', 'can',
       #'floor', 'for', 'from', 'full', 'great', 'has', 'have', 'in', 'is',
       #'it', 'kitchen', 'living', 'located', 'manhattan', 'my',
       #'neighborhood', 'new', 'of', 'on', 'one', 'or', 'park', 'private',
       #'restaurants', 'room', 'space', 'subway', 'that', 'the', 'there',
       #'this', 'to', 'train', 'two', 'very', 'walk', 'we', 'will', 'with',
       #'you', 'your'], dtype=object)
    
#df[df["description"].str.contains("best",regex=True)]["description"]

In [38]:
# lookiga t the description col
df["description"].fillna("",inplace=True)
df[df["description"].str.contains("Luxury|luxury|Luxurious|luxurious|Lux|LUX",regex=True)]["price"].count()
df[df["description"].str.contains("private|Private|Pvt",regex=True)]["price"].count()
df[df["description"].str.contains("spacious|Spacious|Large|large",regex=True)]["price"].count()
df[df["description"].str.contains("cozy|Cozy",regex=True)]["price"].count()
df[df["description"].str.contains("kitchen|Kitchen",regex=True)]["price"].count()
df[df["description"].str.contains("restaurants|Restaurants",regex=True)]["price"].count()
df[df["description"].str.contains("train|Train",regex=True)]["price"].count()
df[df["description"].str.contains("subway|Subway",regex=True)]["price"].count()
df[df["description"].str.contains("park|Park",regex=True)]["price"].count()
df[df["description"].str.contains("cheap|Cheap",regex=True)]["price"].count()
df[df["description"].str.contains("clean|Clean",regex=True)]["price"].count()
df[df["description"].str.contains("best|Best",regex=True)]["price"].count()
df[df["description"].str.contains("expensive|Expensive",regex=True)]["price"].count()

190

In [ ]:
v = CountVectorizer(max_features=30)
v.fit_transform(df["name"])
v.get_feature_names_out()

# array(['and', 'apartment', 'apt', 'beautiful', 'bed', 'bedroom',
       #'brooklyn', 'cozy', 'east', 'heart', 'home', 'in', 'large', 'loft',
       #'luxury', 'manhattan', 'near', 'nyc', 'of', 'park', 'private',
       #'room', 'spacious', 'studio', 'sunny', 'the', 'to', 'village',
       #'williamsburg', 'with'], dtype=object)

In [39]:
# looking at the name col

df["name"].fillna("",inplace=True)
df["name_lux"]=df["name"].str.contains("Luxury|luxury|Luxurious|luxurious|Lux|LUX",regex=True)
df["name_private"]=df["name"].str.contains("private|Private|Pvt",regex=True)
df["name_large"]=df["name"].str.contains("spacious|Spacious|Large|large",regex=True)
df["name_cozy"]=df["name"].str.contains("cozy|Cozy",regex=True)
df["name_beautiful"]=df["name"].str.contains("beautiful|Beautiful",regex=True)
df["name_duplex"]=df["name"].str.contains("Duplex|duplex",regex=True)
#df[df["name"].str.contains("family|FAMILY",regex=True)]["price"].count()

- name/description --> count vextorizer and create new cols
- creat new col using "transit" --> have trans info or not 
- room_type - ohe
- host_response_time -ohe
- host_is_superhost -ohe
- host_neighbourhood - ohe with ma feature 5
    - Williamsburg             2479
    - Bedford-Stuyvesant       1950
    - Bushwick                 1432
    - Hell's Kitchen           1269
    - Upper East Side          1172
- property_type  -  - ohe with ma feature 5
    - Apartment                 27060
    - House                      2471
    - Townhouse                  1086
    - Loft                       1035
    - Condominium                 822
- instant_bookable - ohe
- cancellation_policy - - ohe with ma feature 3
    - strict_14_with_grace_period    15330
    - flexible                       10345
    - moderate                        7746

# Data Processing

In [18]:
# take out the cols i need
numeric_col=["review_scores_rating","reviews_per_month","number_of_reviews","accommodates",
            "bathrooms","bedrooms","beds","review_scores_cleanliness","guests_included","room_type",
             "host_response_time","instant_bookable","host_is_superhost","property_type",
             "host_neighbourhood","cancellation_policy","transit","review_scores_location",
             "host_since","name","amenities",'description']


In [40]:
# write function to help impute missig data 
def impute_mean(df,col):
    cal_mean=df[col].mean()
    df[col].fillna(cal_mean,inplace=True)

def impute_median(df,col):
    cal_med=df[col].median()
    df[col].fillna(cal_med,inplace=True)
    
def impute_zero(df,col):
    df[col].fillna(0,inplace=True)

In [ ]:
# manipulating differnet features

In [22]:
def add_new_col(df):
    df["have_transportation"]=df["transit"].isna()==False
    df=df.drop(columns=["transit"],inplace=True)

In [23]:
def add_new_col2(df):
    df["host_since"].fillna(df["host_since"].mode()[0],inplace=True)
    df["host_since_year"]=[int(i) for i in df["host_since"].str.split("-").str[0]]
    df=df.drop(columns=["host_since"],inplace=True)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

def new_name_feature(df):
    df["name"].fillna("",inplace=True)
    df["name_lux"]=df["name"].str.contains("Luxury|luxury|Luxurious|luxurious|Lux|LUX",regex=True)
    df["name_private"]=df["name"].str.contains("private|Private|Pvt",regex=True)
    df["name_large"]=df["name"].str.contains("spacious|Spacious|Large|large",regex=True)
    df["name_cozy"]=df["name"].str.contains("cozy|Cozy",regex=True)
    df["name_beautiful"]=df["name"].str.contains("beautiful|Beautiful",regex=True)
    df["name_duplex"]=df["name"].str.contains("Duplex|duplex",regex=True)
    df=df.drop(columns=["name"],inplace=True)

In [25]:
def new_des_feature(df):
    df["description"].fillna("",inplace=True)
    df["des_lux"]=df["description"].str.contains("Luxury|luxury|Luxurious|luxurious|Lux|LUX",regex=True)
    df["des_pvt"]=df["description"].str.contains("private|Private|Pvt",regex=True)
    df["des_space"]=df["description"].str.contains("spacious|Spacious|Large|large",regex=True)
    df["des_cozy"]=df["description"].str.contains("cozy|Cozy",regex=True)
    df["des_kitchen"]=df["description"].str.contains("kitchen|Kitchen",regex=True)
    df["des_rest"]=df["description"].str.contains("restaurants|Restaurants",regex=True)
    df["des_train"]=df["description"].str.contains("train|Train",regex=True)
    df["des_sub"]=df["description"].str.contains("subway|Subway",regex=True)
    df["des_park"]=df["description"].str.contains("park|Park",regex=True)
    df["des_cheap"]=df["description"].str.contains("cheap|Cheap",regex=True)
    df["des_clean"]=df["description"].str.contains("clean|Clean",regex=True)
    df["des_best"]=df["description"].str.contains("best|Best",regex=True)
    df=df.drop(columns=["description"],inplace=True)


In [26]:
def num_am_provided(df):
    df["num_am_provided"]=df["amenities"].str.split(",")
    df["num_am_provided"]=df["num_am_provided"].apply(lambda x: len(x))
    df=df.drop(columns=["amenities"],inplace=True)


In [ ]:
df_X=df[numeric_col]
df_Y=df["price"]

In [27]:
# transform original data
impute_mean(df_X,"review_scores_rating")
impute_zero(df_X,"reviews_per_month")
impute_median(df_X,"bedrooms")
impute_median(df_X,"bathrooms")
impute_median(df_X,"beds")
impute_mean(df_X,"review_scores_cleanliness")
impute_mean(df_X,"review_scores_location")
add_new_col(df_X)
add_new_col2(df_X)
new_name_feature(df_X)
num_am_provided(df_X)
new_des_feature(df_X)
df_X.columns

Index(['review_scores_rating', 'reviews_per_month', 'number_of_reviews',
       'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'review_scores_cleanliness', 'guests_included', 'room_type',
       'host_response_time', 'instant_bookable', 'host_is_superhost',
       'property_type', 'host_neighbourhood', 'cancellation_policy',
       'review_scores_location', 'have_transportation', 'host_since_year',
       'name_lux', 'name_private', 'name_large', 'name_cozy', 'name_beautiful',
       'name_duplex', 'num_am_provided', 'des_lux', 'des_pvt', 'des_space',
       'des_cozy', 'des_kitchen', 'des_rest', 'des_train', 'des_sub',
       'des_park', 'des_cheap', 'des_clean', 'des_best'],
      dtype='object')

## Building a pipeline

In [33]:
X_train, X_vali, y_train, y_vali = train_test_split(df_X, df_Y, train_size= 0.8, test_size=0.2)

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Binarizer
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb

In [358]:

pre_col = ColumnTransformer(
    remainder="passthrough",
    transformers=[('normal_onehot', OneHotEncoder(handle_unknown='ignore', dtype='int'),["room_type","host_response_time","instant_bookable"]),
                  ('normal_onehot2', OneHotEncoder(max_categories=2, handle_unknown='ignore', dtype='int'),["host_is_superhost"]),
                  ('normal_onehot3', OneHotEncoder(handle_unknown='ignore', dtype='int'),["property_type","host_neighbourhood"]),
                  ('normal_onehot4', OneHotEncoder(handle_unknown='ignore', dtype='int', max_categories=3),["cancellation_policy"]),
                  ('normal_onehot5', OneHotEncoder(handle_unknown='ignore', dtype='int', max_categories=7),["host_since_year"]),
                  ('binarizer1', Binarizer(threshold=9.6),["review_scores_location"]),
                   ])

pl= Pipeline([
    ("pre_col", pre_col),
    ('rf', RandomForestRegressor(max_depth= 17,
                                 max_features="sqrt",
                                 min_samples_split=8,
                                 n_estimators=400))
])

# mdepth=5, feature=squrt, n est=300
pl_2= Pipeline([
    ("pre_col", pre_col),
    ('gb', GradientBoostingRegressor(criterion='squared_error',
                                     max_depth=16,
                                     max_features="sqrt",
                                     min_samples_leaf=18,
                                     n_estimators=300))
])

pl_3=Pipeline([
    ("pre_col", pre_col),
    ('xgb', XGBRegressor(max_depth=8,
                        n_estimators=150,
                        gamma=0.5,
                        learning_rate=0.1))
])


#{'xgb__gamma': 0.5,
 #'xgb__learning_rate': 0.1,
 #'xgb__max_depth': 8,
 #'xgb__n_estimators': 150}


pl_4=Pipeline([
    ("pre_col", pre_col),
    ('lgb', lgb.LGBMRegressor(learning_rate=0.08,
                              max_depth=23,
                              n_estimators=400,
                              num_leaves=31
                             ))
])

# grid search results for pipeline 4

#{'lgb__learning_rate': 0.08,
 #'lgb__max_depth': 23,
 #'lgb__n_estimators': 400,
 #'lgb__num_leaves': 31}

# {'lgb__learning_rate': 0.08, 
#'lgb__max_depth': 25,
#'lgb__n_estimators': 400}

from sklearn.ensemble import StackingRegressor

estimators = [
    ('gb', pl_2),
    ('llgb', pl_4),   
]

reg=StackingRegressor(estimators)


In [359]:
estimator = [
    ('rf', pl),
    ('gb', pl_2),
    ('xgb', pl_3),
    ('llgb', pl_4),   
]

In [363]:
# 4 pipelines and reg is the one that uses the stacking technique
reg.fit(df_X,df_Y)
pl.fit(df_X,df_Y)
pl_2.fit(df_X,df_Y)
pl_3.fit(df_X,df_Y)
pl_4.fit(df_X,df_Y)


StackingRegressor(estimators=[('gb',
                               Pipeline(steps=[('pre_col',
                                                ColumnTransformer(remainder='passthrough',
                                                                  transformers=[('normal_onehot',
                                                                                 OneHotEncoder(dtype='int',
                                                                                               handle_unknown='ignore'),
                                                                                 ['room_type',
                                                                                  'host_response_time',
                                                                                  'instant_bookable']),
                                                                                ('normal_onehot2',
                                                                                 OneHotEncoder(dtype='int',
                                                                                               handle_unknown='ignore',
                                                                                               max_categories=2),
                                                                                 ['host_is_superhost'])...
                                                                                ('normal_onehot4',
                                                                                 OneHotEncoder(dtype='int',
                                                                                               handle_unknown='ignore',
                                                                                               max_categories=3),
                                                                                 ['cancellation_policy']),
                                                                                ('normal_onehot5',
                                                                                 OneHotEncoder(dtype='int',
                                                                                               handle_unknown='ignore',
                                                                                               max_categories=7),
                                                                                 ['host_since_year']),
                                                                                ('binarizer1',
                                                                                 Binarizer(threshold=9.6),
                                                                                 ['review_scores_location'])])),
                                               ('lgb',
                                                LGBMRegressor(learning_rate=0.08,
                                                              max_depth=23,
                                                              n_estimators=400))]))])

# GRID SEARCH

In [364]:
from sklearn.model_selection import GridSearchCV
# grid search parameters
hyperparams = {
    'rf__max_depth': [10,15,16,17],
    'rf__min_samples_split': [8,12,16,18],
    'rf__n_estimators':[400,500],
    'rf__max_features':["sqrt","log2"],   
}


hyperparams_2={
    'final_estimator__max_depth': [8,10],
    'final_estimator__n_estimators':[300,400],
    'final_estimator__min_samples_leaf':[12,18],
    'final_estimator__max_features':["sqrt"],
}


hyperparams_3={
    'xgb__max_depth': [6,8,10,15],
    'xgb__learning_rate': [0.05,0.1], 
    'xgb__n_estimators':[100,150,300],
}


hyperparams_4 = {
    'final_estimator__max_depth': [15,17,19],
    'final_estimator__min_samples_split': [8,16,18],
    'final_estimator__n_estimators':[300,400],
    'final_estimator__max_features':["sqrt","log2"], 
}


hp_for_pl4= {
    'lgb__max_depth': [20,23,25,30,35], 
    'lgb__learning_rate': [0.05,0.06,0.08,0.1], 
    'lgb__n_estimators': [200,300,400], 
    'lgb__num_leaves': [28,31,35],
}


hyperparams_55 = {
    'final_estimator__max_depth': [25,30,35,40],
    }



hy_pl_5 = {
    'nn__hidden_layer_sizes': [45,100], 
    'nn__max_iter': [150,200,300], 
    
    }
#'lgb__min_samples_split': [8,16,18],
    #'lgb__n_estimators':[300,400],
    #'lgb__max_features':["sqrt","log2"], 

In [365]:
#grid_lol = GridSearchCV(pl_3, hyperparams_3, cv=5,verbose=10)
#grid_lol.fit(df_X, df_Y)

In [366]:
# some of the grid search results

#grid_lol.best_params_
#{'rf__ccp_alpha': 0.01,
# 'rf__max_depth': 15,
 #'rf__min_samples_split': 18,
 #'rf__n_estimators': 400}

#######
#{'rf__max_depth': 17,
 #'rf__max_features': 'sqrt',
# 'rf__min_samples_split': 8,
 #'rf__n_estimators': 400}
    
######
#{'xgb__gamma': 0.5,
 #'xgb__learning_rate': 0.1,
# 'xgb__max_depth': 8,
 #'xgb__n_estimators': 150}

# FINAL PREDICTOR

In [ ]:
pre_col = ColumnTransformer(
    remainder="passthrough",
    transformers=[('normal_onehot', OneHotEncoder(handle_unknown='ignore', dtype='int'),["room_type","host_response_time","instant_bookable"]),
                  ('normal_onehot2', OneHotEncoder(max_categories=2, handle_unknown='ignore', dtype='int'),["host_is_superhost"]),
                  ('normal_onehot3', OneHotEncoder(handle_unknown='ignore', dtype='int'),["property_type","host_neighbourhood"]),
                  ('normal_onehot4', OneHotEncoder(handle_unknown='ignore', dtype='int', max_categories=3),["cancellation_policy"]),
                  ('normal_onehot5', OneHotEncoder(handle_unknown='ignore', dtype='int', max_categories=7),["host_since_year"]),
                  ('binarizer1', Binarizer(threshold=9.6),["review_scores_location"]),
                   ])

pl_2= Pipeline([
    ("pre_col", pre_col),
    ('gb', GradientBoostingRegressor(criterion='squared_error',
                                     max_depth=16,
                                     max_features="sqrt",
                                     min_samples_leaf=18,
                                     n_estimators=300))
])


pl_4=Pipeline([
    ("pre_col", pre_col),
    ('lgb', lgb.LGBMRegressor(learning_rate=0.08,
                              max_depth=23,
                              n_estimators=400,
                              num_leaves=31
                             ))
])


estimators = [
    ('gb', pl_2),
    ('llgb', pl_4),   
]

reg=StackingRegressor(estimators)

# FINAL PREDICTION

In [367]:
kaggle=pd.read_csv("test.csv",low_memory=False)

In [368]:
Ka_X=kaggle[numeric_col]

In [369]:
impute_mean(Ka_X,"review_scores_rating")
impute_zero(Ka_X,"reviews_per_month")
impute_median(Ka_X,"bedrooms")
impute_median(Ka_X,"bathrooms")
impute_median(Ka_X,"beds")
impute_mean(Ka_X,"review_scores_cleanliness")
impute_mean(Ka_X,"review_scores_location")

add_new_col(Ka_X)
add_new_col2(Ka_X)
new_name_feature(Ka_X)
num_am_provided(Ka_X)
new_des_feature(Ka_X)

In [41]:
#reg.predict(Ka_X),reg.predict(Ka_X).mean()

In [371]:
#pl.predict(Ka_X),pl.predict(Ka_X).mean()

In [372]:
#pl_2.predict(Ka_X),pl_2.predict(Ka_X).mean()

In [373]:
#pl_3.predict(Ka_X),pl_3.predict(Ka_X).mean()

In [374]:
#pl_4.predict(Ka_X),pl_4.predict(Ka_X).mean()

In [375]:
#pl_5.predict(Ka_X),pl_5.predict(Ka_X).mean()

In [376]:
b=pd.DataFrame()
b["Id"]=kaggle["id"]
b["Predicted"]=reg.predict(Ka_X)
b

,Id,Predicted
0,19307997,346.492641
1,20176193,175.731615
2,19485371,51.067154
3,13079990,49.045270
4,22339757,63.925625
...,...,...
17332,22325617,90.378145
17333,8372650,377.810030
17334,3812554,87.068167
17335,18891508,68.315615


In [377]:
#reg.predict(Ka_X).mean()

In [378]:
df["price"].mean()

145.177291430616

In [357]:
# export predictions to a csv file
b.to_csv('version_55,pl2pl4_agian.csv',index=False)

# Cross Validation

In [380]:
from sklearn.model_selection import cross_val_score
cross_val_score(reg, df_X, df_Y, cv=5,scoring="neg_mean_squared_error")

array([-8724.57286284, -7250.53246691, -6940.40849295, -7762.15744305,
       -8631.09152636])

In [381]:
a = np.mean([-8724.57286284, -7250.53246691, -6940.40849295, -7762.15744305,
       -8631.09152636])
#7841

-7861.752558422

In [382]:
np.sqrt(7861)

#88.64536084872124
#88.49858755935034

88.66228059327146

In [ ]:
#  recording some of the past versions and their performace:

In [ ]:
#90.37145567047152
#reg: 91.04394543296111
#pl_1: 96.46,96.45 
#pl_2: 94 89
#pl_3:92 91.5
# reg 1616, reduce redendent, add features:
# 20,12,400 95

In [ ]:
#35 (17 sqrt 8 400) 96.67
#36 (dont remember but) 96.51
#37 (17 sqrt 8 400) 96.64
#38 (16 sqrt 16 400) 96.64
#39 (add feature, reduce redundent, 20 sqrt 16 300) -cv:91.04 -kaggle: 94.5
#40 (add feature, reduce redundent, 16 sqrt 16 300) -cv:90.52 -kaggle: 94.85
#41 (40, pl_1 only to 20,12,400) -cv: 90.53 - mean:144.7/145.17 -kaggle: 94.76
#42 lgb light depth= 25 -cv: 90.53 - mean:144.7/145.17 -kaggle: 94.76
#43 version 42 but with stack
#44 norf,nofinal stack
#45 lgbm final stack
#46 gradient boost with greid search
#$& stack gradient boost and lightlgb

- Baseline linear Regression model: 108.42012
- Final model RMSE: 84.75782